In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import sklearn
import sklearn.model_selection
from tensorflow import keras
import keras_tuner as kt

2022-12-19 08:38:29.455746: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
df = pd.read_csv("/home/jordan/Downloads/Titanic/train.csv")
logdir = '/home/jordan/Downloads/Titanic/'

In [3]:
df2 = df.copy()
#X = df2.dropna()
y = df2['Survived']


In [4]:
features = ['Pclass', 'Sex', 'SibSp', 'Parch', 'Embarked']
train_data = pd.get_dummies(df2[features], dummy_na=True)

In [5]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(train_data, y, test_size = 0.2)

## Custom model

def get_model():
        #Custom designed model
    model = tf.keras.Sequential([
            tf.keras.layers.Dense(20, activation='tanh'),
            tf.keras.layers.Dropout(0.4),
            tf.keras.layers.Dense(1, activation='sigmoid')])
    model.compile(optimizer='adam',
            loss='BinaryCrossentropy',
            metrics='binary_accuracy')
    return model

model = get_model()       
history = model.fit(X_train, y_train, epochs=5, batch_size=128)

test_loss, test_acc = model.evaluate(X_test, y_test)
test_acc

## TensorFlow Tuner

In [15]:
from tensorboard.plugins.hparams import api as hp
%load_ext tensorboard
#rm -rf ./logs/

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [16]:
rm -rf ./logs/

In [14]:
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([8, 12, 16, 20, 24, 28]))
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.1, 0.4))
HP_DENSE_LAYERS = hp.HParam("dense_layers", hp.IntInterval(1, 3))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))

METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
  hp.hparams_config(
    hparams=[HP_NUM_UNITS, HP_DROPOUT, HP_OPTIMIZER],
    metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
  )

In [9]:
def auto_tune_model(hparams):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(hparams[HP_NUM_UNITS]),
        tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
        tf.keras.layers.Dense(1, activation = 'sigmoid')
    ])
    model.compile(
    optimizer = hparams[HP_OPTIMIZER],
    loss = 'BinaryCrossentropy',
    metrics = ['accuracy']
    )
    model.fit(X_train, y_train, epochs = 10)
    _, accuracy = model.evaluate(X_test, y_test)
    return accuracy

In [10]:
def run(run_dir, hparams):
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)
        accuracy = auto_tune_model(hparams)
        tf.summary.scalar('accuracy', accuracy, step=1)

In [11]:
session_num = 0

for num_units in HP_NUM_UNITS.domain.values:
  for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
    for optimizer in HP_OPTIMIZER.domain.values:
      hparams = {
          HP_NUM_UNITS: num_units,
          HP_DROPOUT: dropout_rate,
          HP_DENSE_LAYERS: dense_layers,
          HP_OPTIMIZER: optimizer,
      }
      run_name = "run-%d" % session_num
      print('--- Starting trial: %s' % run_name)
      print({h.name: hparams[h] for h in hparams})
      run('logs/hparam_tuning/' + run_name, hparams)
      session_num += 1

--- Starting trial: run-0
{'num_units': 8, 'dropout': 0.1, 'optimizer': 'adam'}
Epoch 1/10
23/23 [==============================] - 1s 4ms/step - loss: 0.9771 - accuracy: 0.4298
Epoch 2/10
23/23 [==============================] - 0s 7ms/step - loss: 0.8212 - accuracy: 0.4902
Epoch 3/10
23/23 [==============================] - 0s 6ms/step - loss: 0.7251 - accuracy: 0.5801
Epoch 4/10
23/23 [==============================] - 0s 10ms/step - loss: 0.6399 - accuracy: 0.7022
Epoch 5/10
23/23 [==============================] - 0s 4ms/step - loss: 0.6276 - accuracy: 0.7205
Epoch 6/10
23/23 [==============================] - 0s 4ms/step - loss: 0.5839 - accuracy: 0.7303
Epoch 7/10
23/23 [==============================] - 0s 3ms/step - loss: 0.5906 - accuracy: 0.7177
Epoch 8/10
23/23 [==============================] - 0s 4ms/step - loss: 0.5615 - accuracy: 0.7374
Epoch 9/10
23/23 [==============================] - 0s 3ms/step - loss: 0.5596 - accuracy: 0.7542
Epoch 10/10
6/6 [====================

In [12]:
!kill 25302

/bin/bash: line 1: kill: (25302) - No such process


In [13]:
%tensorboard --logdir logs/hparam_tuning

## Keras Tuner

def model_builder(hp):
    #Hyperparameter tuned model
    model = keras.Sequential()
    hp_units = hp.Int('units', min_value=8, max_value = 64, step = 4)
    model.add(keras.layers.Dense(units=hp_units, activation='relu'))
    model.add(keras.layers.Dense(1))
              
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss='BinaryCrossentropy',
                  metrics=['binary_accuracy'])
    return model

tuner = kt.Hyperband(model_builder, 
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='/home/jordan/Downloads/Titanic/',
                     project_name='Titanic_NN_tuned')

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
tuner.search(X_train, y_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

best_hps = tuner.get_best_hyperparameters(num_trials=1)

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

print(best_hps)